# Deep Features Extraction

## Collect Extracted Features First model

In [1]:
historyfilename='model/history_model_norm.json'
modelweights='model/model_weights_norm.h5'
thevars="model/MondayDesneNet169/extractedfeatures_norm.pickle"

import pickle

#with open("densenet121/extractedfeatures.pickle", "wb") as f:
#    pickle.dump((Train_Features,Train_Target,Val_Features,Val_Target,Test_Features,Test_Target,model,model_extracted), f)

with open(thevars, "rb") as f:
    Train_Features_A,Train_Target_A,Val_Features_A,Val_Target_A,Test_Features_A,Test_Target_A,model_A,model_extracted_A = pickle.load(f) 

C:\Users\AliHaidar\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\AliHaidar\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## Import Extracted Features Second Model

In [2]:
historyfilename='model/history_model.json'
modelweights='model/model_weights.h5'
thevars="model/MondayDesneNet169/extractedfeatures.pickle"

import pickle

#with open("densenet121/extractedfeatures.pickle", "wb") as f:
#    pickle.dump((Train_Features,Train_Target,Val_Features,Val_Target,Test_Features,Test_Target,model,model_extracted), f)

with open(thevars, "rb") as f:
    Train_Features_B,Train_Target_B,Val_Features_B,Val_Target_B,Test_Features_B,Test_Target_B,model_B,model_extracted_B = pickle.load(f) 

## Combine the Features

In [14]:
import pandas as pd

In [3]:
import numpy as np
Train_Features=np.concatenate([Train_Features_A,Train_Features_B],axis=1)
Train_Target=Train_Target_A

In [4]:
Val_Features=np.concatenate([Val_Features_A,Val_Features_B],axis=1)
Val_Target=Val_Target_A

In [10]:
Test_Features=np.concatenate([Test_Features_A,Test_Features_B],axis=1)
Y_test=Test_Target_A

## Accuracy function

In [7]:
from sklearn.metrics import confusion_matrix
def measure_per(y_val,preds,num_classes):
    print(accuracy_score(y_val,preds))
    print(cohen_kappa_score(y_val,preds))

    
    df_cm = pd.DataFrame(cm, range(num_classes),
                              range(num_classes))
    print(df_cm)

In [8]:
num_classes=2
# Creating a function to report confusion metrics
def confusion_metrics (truvalues,predictions):
# save confusion matrix and slice into four pieces
    conf_matrix = confusion_matrix(truvalues, predictions)
    df_cm = pd.DataFrame(conf_matrix, range(num_classes),
                              range(num_classes))
    print(df_cm)
    TP = conf_matrix[1][1]
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    print('True Positives:', TP)
    print('True Negatives:', TN)
    print('False Positives:', FP)
    print('False Negatives:', FN)
    
    # calculate accuracy
    conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
    
    # calculate mis-classification
    conf_misclassification = 1- conf_accuracy
    
    # calculate the sensitivity
    conf_sensitivity = (TP / float(TP + FN))
    # calculate the specificity
    conf_specificity = (TN / float(TN + FP))
    
    # calculate precision
    conf_precision = (TN / float(TN + FP))
    # calculate f_1 score
    conf_f1 = 2 * ((conf_precision * conf_sensitivity) / (conf_precision + conf_sensitivity))
    print('-'*50)
    print(f'Accuracy: {round(conf_accuracy,2)}') 
    print(f'Mis-Classification: {round(conf_misclassification,2)}') 
    print(f'Sensitivity: {round(conf_sensitivity,2)}') 
    print(f'Specificity: {round(conf_specificity,2)}') 
    print(f'Precision: {round(conf_precision,2)}')
    print(f'f_1 Score: {round(conf_f1,2)}')

## XGBoost

In [ ]:
from sklearn import preprocessing

In [52]:
from xgboost import XGBClassifier,XGBRegressor
from xgboost import plot_importance,plot_tree
eval_set = [(Val_Features, Val_Target.ravel())]

objective='binary:logistic'
eval_metric=["aucpr",'auc']


xgb = XGBClassifier(objective=objective,
                   n_estimators=1000,subsample=1,
                   max_depth=15,n_jobs=3)#binary:logistic

xgb.fit(Train_Features, Train_Target.ravel(),eval_metric=eval_metric, eval_set=eval_set,verbose=False,early_stopping_rounds=50)

preds_xgb = xgb.predict(Test_Features)

from sklearn.metrics import accuracy_score,f1_score,roc_curve,auc
fpr, tpr, thresholds = roc_curve(Y_test, xgb.predict_proba(Test_Features)[:,1])
print("AUC: " + str(auc(fpr, tpr)))

AUC: 0.8952866861030127


In [51]:
confusion_metrics(Y_test,preds_xgb)

     0   1
0  101   4
1   33  65
True Positives: 65
True Negatives: 101
False Positives: 4
False Negatives: 33
--------------------------------------------------
Accuracy: 0.82
Mis-Classification: 0.18
Sensitivity: 0.66
Specificity: 0.96
Precision: 0.96
f_1 Score: 0.79


In [495]:
with open("bestXGBoostmodel.pickle", "wb") as f:
    pickle.dump((xgb), f)

## RF

In [447]:
with open("bestRFmodel/bestRFmodel.pickle", "rb") as f:
    rfa = pickle.load(f) 

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=6, n_estimators=200)
rf.fit(Train_Features, Train_Target)
preds_rf = rf.predict(Test_Features)
fpr, tpr, thresholds = roc_curve(Y_test, rf.predict_proba(Test_Features)[:,1])
print("AUC: " + str(auc(fpr, tpr)))

C:\Users\AliHaidar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


AUC: 0.9102526724975704


In [37]:
confusion_metrics(Y_test,preds_rf)

    0   1
0  89  16
1  18  80
True Positives: 80
True Negatives: 89
False Positives: 16
False Negatives: 18
--------------------------------------------------
Accuracy: 0.83
Mis-Classification: 0.17
Sensitivity: 0.82
Specificity: 0.85
Precision: 0.85
f_1 Score: 0.83


In [468]:
imp_rf = rf.feature_importances_

(1664,)

## Bagging Decision Tree

In [358]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,random_state=0)
bdt = BaggingClassifier(n_estimators=200,base_estimator=clf).fit(Train_Features, Train_Target)
#Predict the response for test dataset
preds_bdt = bdt.predict(Test_Features)

fpr, tpr, thresholds = roc_curve(Y_test, bdt.predict_proba(Test_Features)[:,1])
print("AUC: " + str(auc(fpr, tpr)))

AUC: 0.9015549076773567


In [359]:
confusion_metrics(Y_test,preds_bdt)

    0   1
0  86  19
1  11  87
True Positives: 87
True Negatives: 86
False Positives: 19
False Negatives: 11
--------------------------------------------------
Accuracy: 0.85
Mis-Classification: 0.15
Sensitivity: 0.89
Specificity: 0.82
Precision: 0.82
f_1 Score: 0.85


In [497]:
with open("bestbdtmodel.pickle", "wb") as f:
    pickle.dump((bdt), f)

## Adaboost

In [360]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=200)
clf.fit(Train_Features,Train_Target)
#Predict the response for test dataset
preds_adaboost = clf.predict(Test_Features)

fpr, tpr, thresholds = roc_curve(Y_test, clf.predict_proba(Test_Features)[:,1])
print("AUC: " + str(auc(fpr, tpr)))

AUC: 0.8350826044703596


In [361]:
confusion_metrics(Y_test,preds_adaboost)

    0   1
0  84  21
1  36  62
True Positives: 62
True Negatives: 84
False Positives: 21
False Negatives: 36
--------------------------------------------------
Accuracy: 0.72
Mis-Classification: 0.28
Sensitivity: 0.63
Specificity: 0.8
Precision: 0.8
f_1 Score: 0.71


In [498]:
with open("bestadaboostmodel.pickle", "wb") as f:
    pickle.dump((clf), f)

## GBDT

In [443]:
import lightgbm
train_data = lightgbm.Dataset(Train_Features, label=np.squeeze(Train_Target))
val_data = lightgbm.Dataset(Val_Features, label=np.squeeze(Val_Target))

parameters = {
    'objective': 'cross_entropy',
    'boosting': 'gbdt',#'gbdt' 'dart'
    'num_leaves': 21,
    'feature_fraction': 0.99,
    'bagging_fraction': 0.99,
    'bagging_freq': 30,  'learning_rate': 0.1,
    'verbose': 2,
}

gbm = lightgbm.train(parameters,
                       train_data,
                       valid_sets=val_data,
                       num_boost_round=1000,
                       early_stopping_rounds=100)



[1]	valid_0's xentropy: 0.680922
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's xentropy: 0.662609
[3]	valid_0's xentropy: 0.638034
[4]	valid_0's xentropy: 0.635319
[5]	valid_0's xentropy: 0.622174
[6]	valid_0's xentropy: 0.614138
[7]	valid_0's xentropy: 0.607836
[8]	valid_0's xentropy: 0.603123
[9]	valid_0's xentropy: 0.591247
[10]	valid_0's xentropy: 0.589585
[11]	valid_0's xentropy: 0.585327
[12]	valid_0's xentropy: 0.578837
[13]	valid_0's xentropy: 0.57855
[14]	valid_0's xentropy: 0.581009
[15]	valid_0's xentropy: 0.576708
[16]	valid_0's xentropy: 0.575278
[17]	valid_0's xentropy: 0.573405
[18]	valid_0's xentropy: 0.573272
[19]	valid_0's xentropy: 0.572891
[20]	valid_0's xentropy: 0.575715
[21]	valid_0's xentropy: 0.573019
[22]	valid_0's xentropy: 0.576273
[23]	valid_0's xentropy: 0.576701
[24]	valid_0's xentropy: 0.569728
[25]	valid_0's xentropy: 0.569666
[26]	valid_0's xentropy: 0.567626
[27]	valid_0's xentropy: 0.568559
[28]	valid_0's xentropy: 0.57

In [444]:
preds_gbdt=gbm.predict(Test_Features)
fpr, tpr, thresholds = roc_curve(Y_test, preds_gbdt)
print("AUC: " + str(auc(fpr, tpr)))

AUC: 0.9023323615160349


In [445]:
confusion_metrics(Y_test,np.round(preds_gbdt))

    0   1
0  90  15
1  18  80
True Positives: 80
True Negatives: 90
False Positives: 15
False Negatives: 18
--------------------------------------------------
Accuracy: 0.84
Mis-Classification: 0.16
Sensitivity: 0.82
Specificity: 0.86
Precision: 0.86
f_1 Score: 0.84


In [500]:
with open("bestlightgbmmodel.pickle", "wb") as f:
    pickle.dump((gbm), f)

## Dart

In [289]:
import lightgbm
train_data = lightgbm.Dataset(Train_Features, label=np.squeeze(Train_Target))
val_data = lightgbm.Dataset(Val_Features, label=np.squeeze(Val_Target))

parameters = {
    'objective': 'binary',
    'boosting': 'dart',#'gbdt' 'dart'
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'bagging_freq': 30,
    'verbose': 2,
}

dart = lightgbm.train(parameters,
                       train_data,
                       valid_sets=val_data,
                       num_boost_round=1000,
                       early_stopping_rounds=100)



[1]	valid_0's binary_logloss: 0.681957
[2]	valid_0's binary_logloss: 0.671843
[3]	valid_0's binary_logloss: 0.651553
[4]	valid_0's binary_logloss: 0.635626
[5]	valid_0's binary_logloss: 0.620932
[6]	valid_0's binary_logloss: 0.6112
[7]	valid_0's binary_logloss: 0.601755
[8]	valid_0's binary_logloss: 0.602437
[9]	valid_0's binary_logloss: 0.591426
[10]	valid_0's binary_logloss: 0.582238
[11]	valid_0's binary_logloss: 0.565282
[12]	valid_0's binary_logloss: 0.570528
[13]	valid_0's binary_logloss: 0.561578
[14]	valid_0's binary_logloss: 0.559752
[15]	valid_0's binary_logloss: 0.560612
[16]	valid_0's binary_logloss: 0.557724
[17]	valid_0's binary_logloss: 0.560193
[18]	valid_0's binary_logloss: 0.560525
[19]	valid_0's binary_logloss: 0.556087
[20]	valid_0's binary_logloss: 0.552506
[21]	valid_0's binary_logloss: 0.554106
[22]	valid_0's binary_logloss: 0.556701
[23]	valid_0's binary_logloss: 0.550505
[24]	valid_0's binary_logloss: 0.557925
[25]	valid_0's binary_logloss: 0.558272
[26]	valid_

[403]	valid_0's binary_logloss: 0.905057
[404]	valid_0's binary_logloss: 0.909214
[405]	valid_0's binary_logloss: 0.917771
[406]	valid_0's binary_logloss: 0.915099
[407]	valid_0's binary_logloss: 0.922397
[408]	valid_0's binary_logloss: 0.919255
[409]	valid_0's binary_logloss: 0.92567
[410]	valid_0's binary_logloss: 0.932489
[411]	valid_0's binary_logloss: 0.938688
[412]	valid_0's binary_logloss: 0.935562
[413]	valid_0's binary_logloss: 0.932205
[414]	valid_0's binary_logloss: 0.94035
[415]	valid_0's binary_logloss: 0.948593
[416]	valid_0's binary_logloss: 0.955435
[417]	valid_0's binary_logloss: 0.951389
[418]	valid_0's binary_logloss: 0.947746
[419]	valid_0's binary_logloss: 0.948102
[420]	valid_0's binary_logloss: 0.944977
[421]	valid_0's binary_logloss: 0.941573
[422]	valid_0's binary_logloss: 0.942651
[423]	valid_0's binary_logloss: 0.955305
[424]	valid_0's binary_logloss: 0.963742
[425]	valid_0's binary_logloss: 0.959851
[426]	valid_0's binary_logloss: 0.963506
[427]	valid_0's bi

[812]	valid_0's binary_logloss: 1.20042
[813]	valid_0's binary_logloss: 1.20619
[814]	valid_0's binary_logloss: 1.20452
[815]	valid_0's binary_logloss: 1.20985
[816]	valid_0's binary_logloss: 1.2145
[817]	valid_0's binary_logloss: 1.21249
[818]	valid_0's binary_logloss: 1.21476
[819]	valid_0's binary_logloss: 1.21322
[820]	valid_0's binary_logloss: 1.21104
[821]	valid_0's binary_logloss: 1.21366
[822]	valid_0's binary_logloss: 1.21166
[823]	valid_0's binary_logloss: 1.20987
[824]	valid_0's binary_logloss: 1.21178
[825]	valid_0's binary_logloss: 1.21216
[826]	valid_0's binary_logloss: 1.21063
[827]	valid_0's binary_logloss: 1.20874
[828]	valid_0's binary_logloss: 1.2104
[829]	valid_0's binary_logloss: 1.214
[830]	valid_0's binary_logloss: 1.21308
[831]	valid_0's binary_logloss: 1.21244
[832]	valid_0's binary_logloss: 1.21522
[833]	valid_0's binary_logloss: 1.22094
[834]	valid_0's binary_logloss: 1.21902
[835]	valid_0's binary_logloss: 1.21742
[836]	valid_0's binary_logloss: 1.21904
[837

In [407]:
preds_dart=dart.predict(Test_Features)
fpr, tpr, thresholds = roc_curve(Y_test, preds_dart)
print("AUC: " + str(auc(fpr, tpr)))

AUC: 0.897181729834791


In [408]:
confusion_metrics(Y_test,np.round(preds_dart))

    0   1
0  93  12
1  15  83
True Positives: 83
True Negatives: 93
False Positives: 12
False Negatives: 15
--------------------------------------------------
Accuracy: 0.87
Mis-Classification: 0.13
Sensitivity: 0.85
Specificity: 0.89
Precision: 0.89
f_1 Score: 0.87


In [501]:
with open("bestdartmmodel.pickle", "wb") as f:
    pickle.dump((dart), f)

# Cluster

In [471]:
# import KMeans
from sklearn.cluster import KMeans

In [489]:
# create kmeans object
kmeans = KMeans(n_clusters=3)
# fit kmeans object to data
kmeans.fit(Train_Features)
# print location of clusters learned by kmeans object
print(kmeans.cluster_centers_)

[[1.6134991e-03 2.1081895e-03 0.0000000e+00 ... 1.7568034e-01
  8.0918729e-02 2.6544614e+00]
 [8.0086337e-04 1.2054585e-03 1.5514814e-08 ... 8.4059513e-01
  1.2176312e+00 1.2046247e+00]
 [1.1649001e-03 1.4998619e-03 3.6504311e-08 ... 7.9408073e-01
  4.5360473e-01 1.0286644e+00]]


In [490]:
y_km = kmeans.fit_predict(Train_Features)

In [491]:
y_km

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 2, 2, 2, 2, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,

In [492]:
s = kmeans.fit_predict(Test_Features)

In [493]:
print(s)

[2 1 2 0 2 2 2 2 2 2 2 2 2 2 1 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 0 1 1
 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 2 2 0 2 2 2 0 1 1 2 2 2 2
 0 0 1 0 0 0 0 1 1 1 1 0 1 0 0 2 2 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 1 1 0 2 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1]


In [518]:
np.round(preds_dart)

array([1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.])

# Check Features

In [512]:
import pandas as pd

In [520]:
di={'actual':Y_test,'preds_xgboost':preds_xgb,'preds_rf':preds_rf,
   'preds_bdt':preds_bdt,'preds_adaboost':preds_adaboost,'preds_gbdt':np.round(preds_gbdt),'preds_dart':np.round(preds_dart)}

In [521]:
df=pd.DataFrame(di)

In [522]:
df

,actual,preds_xgboost,preds_rf,preds_bdt,preds_adaboost,preds_gbdt,preds_dart
0,1,1,1,1,1,1.0,1.0
1,1,1,1,1,0,1.0,0.0
2,1,1,1,1,1,1.0,1.0
3,1,1,1,1,1,1.0,1.0
4,1,1,1,1,1,1.0,1.0
5,1,1,1,1,1,1.0,1.0
6,1,1,1,1,1,1.0,1.0
7,1,1,1,1,1,1.0,1.0
8,1,1,1,1,1,1.0,1.0
9,1,1,1,1,0,0.0,1.0
